# Libraries

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import XPSLibrary as xl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Functions

In [ ]:
def Aesthetics(fig):
    
    fig.update_layout(template="plotly_white")
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)')

    fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
    fig.update_layout(font=dict(size=30))
    #add the black box around the graph
    fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    #add black ticks
    fig.update_xaxes(showgrid=True,ticks="outside",tickwidth=4)
    fig.update_yaxes(showgrid=True,ticks="outside",tickwidth=4)
    
    #fig.update_xaxes(autorange = "reversed")
    fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
    fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
    fig.update_layout(autosize=False, width=500, height=500) #w=800, h=600

# Automated code

## Parameters

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Br4TPP/Mica/"+"0020_300621_AQ"
#File of the temperature ramp
FileTR="TemperatureRamp"

#Files with XPS Data
files=["Br3p"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

## Main

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Mask= TemperatureRamp['Time'] <=19.95
#TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0, 1): #len(files)
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra
    #Normalize and print the data. ALl the traces should cross the horizintal line in the same point
    XPS=xl.Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=15)
    #Remove the linear background and print the data
    
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)

    

    print(FileXPS)
    
    ######################################################
    #NOTE THAT I PLOTTED STARTING FORM THE THIRD SPECTRUM#
    ######################################################
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[2:], #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature[2:], #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'RdBu_r'))
    fig.data[0].update(zmin=-200, zmax=500) #(zmin=np.min(XPS["CPS"].values)
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_xaxes(range = (180,188))
    #fig.update_yaxes(range = (60,250))

    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    #fig.write_image(FolderTR+"/"+files[j]+"_60-400"+".pdf")

In [ ]:
for j in range(0, len(files)): 
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    if (FileXPS=="Br3p"):
        Mask=np.logical_and( XPS["Binding Energy"] > 175, XPS["Binding Energy"] < 196)
        XPS=XPS[Mask]

       
        
    if (FileXPS=="Cu2p"):
        Mask=np.logical_and( XPS["Binding Energy"] > 925, XPS["Binding Energy"] < 939)
        

        
    
  
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra
    #Normalize and print the data. ALl the traces should cross the horizintal line in the same point
    XPS=xl.Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=10)
    #Remove the linear background and print the data
    if (FileXPS!="Br3p"):
        XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    

    print(FileXPS)
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_xaxes(range = (180,196))
    fig.update_yaxes(range = (35,250))

    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    #fig.write_image(FolderTR+"/"+files[j]+"spline_250"+".png")

## Cu5+Br4TPP

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Br4TPP/Mica/"+"0003_070721_AQ"
#File of the temperature ramp
FileTR="TemperatureRamp"

#Files with XPS Data
files=["Cu2p"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Mask= TemperatureRamp['Time'] <=19.95
#TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
for j in range(0, 1): #len(files)
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    #XPS=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)
    
    if (FileXPS == "Br3p_spline"):
        Names=["K.E.","Counts", "Background", "Envelope", "Binding Energy","CPS","Background CPS", "Envelope CPS"]
        XPS=xl.ReadXPSDataFitted(FolderTR, FileXPS, HeaderColumns=8, ColumnsNames=Names)
    else:
        XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)

    
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra
    #Normalize and print the data. ALl the traces should cross the horizintal line in the same point
    #XPS=xl.Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=15)
    #Remove the linear background and print the data
    if (FileXPS != "Br3p"):
        XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #fig = go.Figure()
    #for i in range (0, NumberOfSpectra):
        #fig.add_trace(go.Scatter(mode='markers',
                                 #x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 #y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]))#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len)[i]))
        
    #fig.show()

    print(FileXPS)
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len]+3.5, #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'RdBu_r'))
    #fig.data[0].update(zmin=-200, zmax=475)
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_layout(xaxis = dict(autorange = "reversed"))
    #fig.update_xaxes(range = (180,188))
    #fig.update_yaxes(range = (35,250))

    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    #fig.write_image(FolderTR+"/"+files[j]+"_scale.pdf")

In [ ]:
fig = go.Figure()
fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intesnity (a.u.)')
fig.update_layout(yaxis = dict( tickfont = dict(size = 23)), xaxis = dict( tickfont = dict(size = 23)))
fig.update_layout(font=dict(size=23))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="outside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="outside",tickwidth=4)
    
fig.update_layout(legend=dict(yanchor="top",y=0.89,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)
fig.update_layout(xaxis = dict(autorange = "reversed"))
    
fig.add_vline(x=932.4, line_dash="dash",annotation_text="Cu<sup>+1</sup>", annotation_position="top right")

fig.add_vline(x=935, line_dash="dot",annotation_text="Cu<sup>+2</sup>", annotation_position="top left")

fig.add_trace(go.Scatter(mode='lines', marker=dict(size=12,opacity=0.3),
                                 x=XPS["Binding Energy"][0:x_len],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]/np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]),#+XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[2])/2 ,
                                name="50 <span>&#176;</span>C"))

fig.add_trace(go.Scatter(mode='lines', marker=dict(size=12,opacity=0.3),
                                 x=XPS["Binding Energy"][0:x_len],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[3]/np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[3]),#+XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[2])/2 ,
                                name="100 <span>&#176;</span>C"))

fig.add_trace(go.Scatter(mode='lines', marker=dict(size=12,opacity=0.3, symbol="square"),
                                 x=XPS["Binding Energy"][0:x_len],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[20]/np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[20]),
                                name="400 <span>&#176;</span>C"))

fig.update_xaxes(range = (928,939))

fig.update_layout(showlegend=True)



fig.show()

fig.write_image("C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/"+"Cu.png")

# Previous versions of the code

## Temperature Ramp

In [ ]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0009_100321_AQ" #Folder of the Temperature ramp
FileTR="20210310_TemperatureRamp" #File of the temperature ramp
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Fit the temperature data
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

## XPS Data

In [ ]:
HeaderColumns=8 #number of columns comprising the headers
FileXPS="Au4f" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=169 #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPS=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPS.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSAu=SetTemperatureColumn(XPSAu, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPS.shape[0]//NumberOfSpectra
#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPS=xl.Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)
#Remove the linear background and print the data
XPS=xl.RemoveLinearBackgorund(XPS, Points=30, NumberOfSpectra=NumberOfSpectra)

In [ ]:
fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
#save picture 
#fig.write_image(FolderTR+"/"+FileXPS+".svg") #save picture 

## Code for each peak

### Au

In [ ]:
FileXPS="Au4f" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=169 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSAu=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSAu.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSAu=SetTemperatureColumn(XPSAu, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)

In [ ]:
x_len=XPSAu.shape[0]//NumberOfSpectra

#Print 3 subplot to show each stage of the manipulation: rwad data, normalization and linear background subtraction
fig = make_subplots(rows=3, cols=1, subplot_titles=("Raw Data", "Normalization", "Background Subtraction"), shared_xaxes=False)
                    #Print the traces of the raw data
for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSAu["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPSAu=xl.Normalize(XPSAu,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSAu["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

#Remove the linear background and print the data
XPSAu=xl.RemoveLinearBackgorund(XPSAu, Points=30, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSAu["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)

fig.update_xaxes(title_text="Binding Energy (eV)", row=3, col=1)
fig.update_yaxes(title_text="CPS", row=1, col=1)
fig.update_yaxes(title_text="CPS", row=2, col=1)
fig.update_yaxes(title_text="CPS", row=3, col=1)
    
#fig.write_html(FolderTR+"/"+"Au"+".html")

fig.show()

In [ ]:
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False)#redefine the length of the vectors along the x axis so that the graph works even when masking the data

x_len=XPSAu.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSAu["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)
fig.update_traces(showscale=False) #hides the colorbar
fig.show()
#save picture 
fig.write_image(FolderTR+"/"+"Au.svg") #save picture 

### N

In [ ]:
#FileXPS="BackGroundSubtractionN-Smooth" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
FileXPS="N1s"
DataPerSpectrum=209 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSN=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSN.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSN=SetTemperatureColumn(XPSN, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)

In [ ]:
x_len=XPSN.shape[0]//NumberOfSpectra

#Print 3 subplot to show each stage of the manipulation: rwad data, normalization and linear background subtraction
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

#Print the traces of the raw data
for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSN["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPSN=xl.Normalize(XPSN,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSN["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

#Remove the linear background and print the data
XPSN=xl.RemoveLinearBackgorund(XPSN, Points=30, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSN["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)
    
fig.update_xaxes(title_text="Binding Energy (eV)", row=3, col=1)
fig.update_yaxes(title_text="CPS", row=1, col=1)
fig.update_yaxes(title_text="CPS", row=2, col=1)
fig.update_yaxes(title_text="CPS", row=3, col=1)
    
#fig.write_html(FolderTR+"/"+"N"+".html")

fig.show()

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPSN.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSN["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
fig.write_image(FolderTR+"/"+"N.svg") #save picture 

### Br

In [ ]:
FileXPS="Br3p" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=209 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSBr=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSBr.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSN=SetTemperatureColumn(XPSBr, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
#Temperature=TemperatureVector(regression, NumberOfSpectra)

In [ ]:
x_len=XPSBr.shape[0]//NumberOfSpectra

fig = make_subplots(rows=3, cols=1, subplot_titles=("Raw Data", "Normalization", "Backrgound Subtraction", "Again Normalized"), shared_xaxes=False)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

XPSBr=xl.Normalize(XPSBr,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

XPSBr=xl.RemoveLinearBackgorund(XPSBr, Points=50, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='CPS')


#fig.write_html(FolderTR+"/"+"Br"+".html")

fig.show()

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPSBr.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSBr["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
fig.write_image(FolderTR+"/"+"Br.svg") #save picture 

# Leo's Data

## Temperature Ramp

In [ ]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/LeoTPXPS" #Folder of the Temperature ramp
FileTR="Datalog_ZnTPP_Au111_TP_2" #File of the temperature ramp
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Fit the temperature data
Mask= TemperatureRamp['Temperature'] <= 250
regression = stats.linregress(TemperatureRamp['Time'][Mask], TemperatureRamp['Temperature'][Mask])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'][Mask], y=TemperatureRamp['Time'][Mask]*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

## Br

In [ ]:
HeaderColumns=8 #number of columns comprising the headers
FileXPS="LeoBr" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=489 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSBr=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSBr.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSN=SetTemperatureColumn(XPSBr, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'][Mask], NumberOfSpectra=NumberOfSpectra)

In [ ]:
x_len=XPSBr.shape[0]//NumberOfSpectra

#Print 3 subplot to show each stage of the manipulation: rwad data, normalization and linear background subtraction
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

#Print the traces of the raw data
for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPSBr=xl.Normalize(XPSBr,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

#Remove the linear background and print the data
XPSBr=xl.RemoveLinearBackgorund(XPSBr, Points=30, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)
    
fig.update_xaxes(title_text="Binding Energy (eV)", row=3, col=1)
fig.update_yaxes(title_text="CPS", row=1, col=1)
fig.update_yaxes(title_text="CPS", row=2, col=1)
fig.update_yaxes(title_text="CPS", row=3, col=1)
    
#fig.write_html(FolderTR+"/"+"Br"+".html")

fig.show()

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPSBr.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSBr["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
fig.write_image(FolderTR+"/"+"Br.svg") #save picture 

# Leo's data N1s 19/03/20

In [ ]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0014_190320_SPJ_LF" #Folder of the Temperature ramp
FileTR="Br4TPP_Au111_Cu5_10pc_10min_postdeposited_2" #File of the temperature ramp
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Fit the temperature data
Mask= TemperatureRamp['Temperature'] <= 250
regression = stats.linregress(TemperatureRamp['Time'][Mask], TemperatureRamp['Temperature'][Mask])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'][Mask], y=TemperatureRamp['Time'][Mask]*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
HeaderColumns=8 #number of columns comprising the headers
FileXPS="N1s" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=149 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSBr=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSBr.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSN=SetTemperatureColumn(XPSBr, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'][Mask], NumberOfSpectra=NumberOfSpectra)

In [ ]:
x_len=XPSBr.shape[0]//NumberOfSpectra

#Print 3 subplot to show each stage of the manipulation: rwad data, normalization and linear background subtraction
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

#Print the traces of the raw data
for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPSBr=xl.Normalize(XPSBr,NumberOfSpectra=NumberOfSpectra, NumberOfData=30)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

#Remove the linear background and print the data
XPSBr=xl.RemoveLinearBackgorund(XPSBr, Points=50, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)
    
fig.update_xaxes(title_text="Binding Energy (eV)", row=3, col=1)
fig.update_yaxes(title_text="CPS", row=1, col=1)
fig.update_yaxes(title_text="CPS", row=2, col=1)
fig.update_yaxes(title_text="CPS", row=3, col=1)
    
#fig.write_html(FolderTR+"/"+"N1s"+".html")

fig.show()

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPSBr.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSBr["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
#fig.write_image(FolderTR+"/"+"N.svg") #save picture 

# Br$_4$TPP on Mica

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"0022_310321_AQLF"
#File of the temperature ramp
FileTR="TemperaureRamp_2021_03_31"

#Files with XPS Data
files=["N1s","C1s","Br3p","Br3d"]
#Number of data per spectrum
DataPerSpectra=[169,239,309,149]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <= 24
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0, len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)#, DataPerSpectrum, HeaderColumns)

    #number of spectra composing the dataset
    NumberOfSpectra=(XPS.shape[0]//NumericDataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra
    #Normalize and print the data. ALl the traces should cross the horizintal line in the same point
    XPS=xl.Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=15)
    #Remove the linear background and print the data
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
    
    print(FileXPS)
        
    NumberOfSpectra=24*4
    XPS=XPS[0:NumberOfSpectra*NumericDataPerSpectrum]
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)

    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
   # fig.write_image(FolderTR+"/"+FileXPS+".svg")

In [ ]:
FileXPS=files[2] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=DataPerSpectra[2] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPS=xl.ReadXPSData(FolderTR, FileXPS)#, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPS.shape[0]//NumericDataPerSpectrum)                  
    
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPS.shape[0]//NumberOfSpectra
#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPS=xl.Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=15)
#Remove the linear background and print the data
XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)

In [ ]:
XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0], #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
x=XPS["Binding Energy"][0:x_len]


fig = go.Figure()
fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Etching time (s)', yaxis_title='Atomic percentage (%)')
fig.update_layout(yaxis = dict( tickfont = dict(size = 23)), xaxis = dict( tickfont = dict(size = 23)))
fig.update_layout(font=dict(size=23))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="outside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="outside",tickwidth=4)
    
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)
    


fig.add_trace(go.Scatter(mode='markers+lines', marker=dict(size=12,opacity=0.5),
                                 x=XPS["Binding Energy"][0:x_len],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0],
                                name="50 C"))

fig.add_trace(go.Scatter(mode='markers+lines', marker=dict(size=12,opacity=0.5),
                                 x=XPS["Binding Energy"][0:x_len],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[100],
                                name="100 C"))

fig.update_layout(showlegend=True)

fig.show()


# Edward's WS$_2$ on HOPG 

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Edward/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"0008_121022_EJD_AQ"
#File of the temperature ramp
FileTR="Temperature Ramp"

#Files with XPS Data
files=["C 1s","S 2p", "w 4f"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <=13.41
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0, len(files)): #len(files)
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    #XPS=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)
    
   
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)

    
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra
    #Normalize and print the data. ALl the traces should cross the horizintal line in the same point
    XPS=xl.Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=15)
    #Remove the linear background and print the data
    
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
  
    print(FileXPS)    
    
    #normlaise the data so that hte maximum of each spectrum is 1
    for k in range (0, NumberOfSpectra):
        XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[k]=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[k]/np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[k])
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    #colorscale= 'RdBu_r'
    ))
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_xaxes(autorange="reversed")
    #fig.update_xaxes(range = )
    #fig.update_yaxes(range = (35,250))

    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    #fig.write_html(FolderTR+"/"+files[j]+".html")
    #fig.write_image(FolderTR+"/"+files[j]+".png")
    #fig.write_image(FolderTR+"/"+files[j]+".svg")
    
    fig2 = go.Figure()
    fig2.update_layout(template="plotly_white")
    fig2.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised (a.u.)')
    fig2.update_layout(yaxis = dict( tickfont = dict(size = 23)), xaxis = dict( tickfont = dict(size = 23)))
    fig2.update_layout(font=dict(size=23))
    #add the black box around the graph
    fig2.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    fig2.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    #add black ticks
    fig2.update_xaxes(showgrid=True,ticks="outside",tickwidth=4)
    fig2.update_yaxes(showgrid=True,ticks="outside",tickwidth=4)

    fig2.update_layout(legend=dict(yanchor="top",y=0.89,xanchor="right",x=0.99))
    fig2.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
    fig2.update_layout(autosize=False, width=550, height=500)

    #fig.add_vline(x=932.4, line_dash="dash",annotation_text="Cu metal", annotation_position="top left")

    #fig.add_vline(x=934.7, line_dash="dot",annotation_text="Oxidised Cu", annotation_position="top right")

    fig2.add_trace(go.Scatter(mode='lines', line=dict(dash="dash"), marker=dict(size=12,opacity=0.7),
                                     x=XPS["Binding Energy"][0:x_len],
                                     y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0] ,
                                    name="50 <span>&#176;</span>C"))

    fig2.add_trace(go.Scatter(mode='lines', marker=dict(size=12,opacity=0.7, symbol="square"),
                                     x=XPS["Binding Energy"][0:x_len],
                                     y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[NumberOfSpectra-1],
                                    name="450 <span>&#176;</span>C"))

    fig2.update_xaxes(autorange="reversed")
    #fig2.update_xaxes(range = (928,939))
    
    fig2.update_layout(showlegend=True)

    fig2.show()

    #fig2.write_html(FolderTR+"/"+files[j]+"spectra "+".html")
    #fig2.write_image(FolderTR+"/"+files[j]+"spectra "+".png")
    #fig2.write_image(FolderTR+"/"+files[j]+"spectra "+".svg")
    
    DE=XPS["Binding Energy"][0:x_len][np.argmax(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0])] - XPS["Binding Energy"][0:x_len][np.argmax(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[NumberOfSpectra-1])]

    print ("Energy shift: ", np.abs(DE))
    

# Reply to referees

## N 1s up to 400 C

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Br4TPP/Mica/Reply to referees/"+"0000_011122_AQ"
#File of the temperature ramp
FileTR="Temperature Ramp"

#Files with XPS Data
files=["Al 2p", "C 1s", "N 1s", "Br 3p"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Temperature'] <=15.65
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    #XPS=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)
    
    if (FileXPS == "Br 3p"):
        XPS=xl.ReadXPSData(FolderTR, FileXPS)
        
    else:
        XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    
    if (FileXPS == "Br 3p"):
        mask= np.logical_and(XPS["Binding Energy"]<185, XPS["Binding Energy"]>180)
        XPS=XPS[mask]
        XPS=xl.RemoveLinearBackgorund(XPS, Points=5, NumberOfSpectra=NumberOfSpectra)
    else:
        XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'RdBu_r'))
    #fig.data[0].update(zmin=-200, zmax=475)
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_layout(xaxis = dict(autorange = "reversed"))


    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")

## HOPG

### 225 C

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Br4TPP/HOPG/Reply to referees/"+"0006_081122_AQ"
#File of the temperature ramp
FileTR="Temperature Ramp"

#Files with XPS Data
files=["Cu 2p", "C 1s", "Br 3p", "N1s"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <=10
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    #XPS=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)
    
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    
    #if (FileXPS == "Br 3p"):
        #mask= np.logical_and(XPS["Binding Energy"]<185, XPS["Binding Energy"]>180)
        #XPS=XPS[mask]
        #XPS=xl.RemoveLinearBackgorund(XPS, Points=5, NumberOfSpectra=NumberOfSpectra)
    #else:
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'RdBu_'))
    #fig.data[0].update(zmin=-200, zmax=475)
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_layout(xaxis = dict(autorange = "reversed"))


    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")

### 400 C

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Br4TPP/HOPG/Reply to referees/"+"0009_101122_AQ"
#File of the temperature ramp
FileTR="Temperature Ramp"

#Files with XPS Data
files=["Cu 2p", "C 1s", "Br 3p", "N 1s"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <=16.79
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
 
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)
    
        
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'RdBu_r'))
    #fig.data[0].update(zmin=-200, zmax=475)
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_layout(xaxis = dict(autorange = "reversed"))


    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")

# Ag 5

## Powders

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0008_170123_AQ']

### Ag 3d

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0008_170123_AQ']
Files=["Ag 3d"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.4))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Powders_Spectrum_Ag3d"+".svg")
fig.write_image(Path_s+"/"+ "Powders_Spectrum_Ag3d"+".png")
fig.show()

### Ag MNN

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0008_170123_AQ']
Files=["AgMNN"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Kinetic Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=1486.6-XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Powders_Spectrum_AgMNN"+".svg")
fig.write_image(Path_s+"/"+ "Powders_Spectrum_Ag3MNN"+".png")
fig.show()

### Si 2p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0008_170123_AQ']
Files=["Si2p"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Powders_Spectrum_Si 2p"+".svg")
fig.write_image(Path_s+"/"+ "Powders_Spectrum_Si 2p"+".png")
fig.show()

### N 1s

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0008_170123_AQ']
Files=["N1s"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        
        if (i==2):
            
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i]-0.5,
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
            
        else:
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Powders_Spectrum_N 1s"+".svg")
fig.write_image(Path_s+"/"+ "Powders_Spectrum_N 1s"+".png")
fig.show()

## Spin-coated

### Ag 3d

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0012_240123_AQ']
Files=["Ag3d"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.3))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Solution_Spectrum_Ag3d"+".svg")
fig.write_image(Path_s+"/"+ "Solution_Spectrum_Ag3d"+".png")
fig.show()

### Ag MNN

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0012_240123_AQ']
Files=["AgMNN"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Kinetic Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=1486.6-XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Solution_Spectrum_AgMNN"+".svg")
fig.write_image(Path_s+"/"+ "Solution_Spectrum_Ag3MNN"+".png")
fig.show()

### Si 2p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0012_240123_AQ']
Files=["Si2p"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
            XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Solutiom_Spectrum_Si 2p"+".svg")
fig.write_image(Path_s+"/"+ "Solution_Spectrum_Si 2p"+".png")
fig.show()

### N 1s

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/"
Folders=['0012_240123_AQ']
Files=["N1s"]  

names=["NG 14", "NG 15", "NG 21"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):

        XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
        fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Solution_Spectrum_N 1s"+".svg")
fig.write_image(Path_s+"/"+ "Solution_Spectrum_N 1s"+".png")
fig.show()

# Br4TPP and Ag5 Ng15

## Au(111)

### 2023_02_14

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Ag5/Ag5 and Br4TPP/"+"0009_140223_AQ"
#File of the temperature ramp
FileTR="Temperature Ramp"

#Files with XPS Data
files=["N1s", "Br3p", "ag3d", "au4f", "C 1s"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <=18.41
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    #XPS=xl.ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)
    
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    
    #if (FileXPS == "Br 3p"):
        #mask= np.logical_and(XPS["Binding Energy"]<185, XPS["Binding Energy"]>180)
        #XPS=XPS[mask]
        #XPS=xl.RemoveLinearBackgorund(XPS, Points=5, NumberOfSpectra=NumberOfSpectra)
    #else:
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'thermal'))
    #fig.data[0].update(zmin=-200, zmax=475)
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_layout(xaxis = dict(autorange = "reversed"))


    fig.update_traces(showscale=False)#hides the colorbar
    
    fig.update_layout(template="plotly_white")
    #fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
    #fig.update_yaxes(showticklabels=False,title_standoff = 30)
    fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
    fig.update_layout(font=dict(size=30))
    #add the black box around the graph
    fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    #add black ticks
    fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
    fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)

    #fig.update_layout(xaxis = dict(autorange = "reversed"))
    fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.4))
    fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
    fig.update_layout(autosize=False, width=550, height=500)
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")

### 2023_03_23

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Ag5/Ag5 and Br4TPP/"+"0001_230323_AQ"
#File of the temperature ramp
FileTR="TemperatureRamp"

#Files with XPS Data
files=["N 1s", "Br 3p", "Ag 3d", "Au 4f"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Mask= TemperatureRamp['Time'] <=18.41
#TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data

    print(FileXPS)
    
    if (FileXPS=="Ag 3d"):
        Mask=np.logical_and(XPS["Binding Energy"]<371, XPS["Binding Energy"]>365)
        XPS=XPS[Mask]
    
    x_len=XPS.shape[0]//NumberOfSpectra

    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'thermal'))
    #fig.data[0].update(zmin=-200, zmax=475)
###
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_layout(xaxis = dict(autorange = "reversed"))


    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")

### 150 C 28_03_2023

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Ag5/Ag5 and Br4TPP/"+"0003_280323_AQ"
#File of the temperature ramp
FileTR="TemperatureRamp"

#Files with XPS Data
files=["N 1s", "Ag 3d", "Au 4f","Br 3p"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Temperature'] >= 40
TemperatureRamp=TemperatureRamp[Mask]
#TemperatureRamp['Time']=TemperatureRamp['Time']-TemperatureRamp['Time'][TemperatureRamp['Time'].index[0]]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])


In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)    
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data

    print(FileXPS)
    
    x_len=XPS.shape[0]//NumberOfSpectra

    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'thermal'))
    #fig.data[0].update(zmin=-200, zmax=475)
###
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    fig.update_layout(xaxis = dict(autorange = "reversed"))


    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")

### Br 3p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/Ag5 and Br4TPP/"
Folders=['0001_230323_AQ']
Files=["Br 3p_RT_150"]  

names=["30 °C", "150 °C"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.4))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
        
    x_len=XPS.shape[0]//NumberOfSpectra
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)
    XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])

    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        
        if (i==1):
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]))/3,
                                 name=names[i]))
        else:
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Au(111)_Spectrum_Br_RT_150C_400C"+".svg")
fig.write_image(Path_s+"/"+ "Au(111)_Spectrum_Br_RT_150C_400C"+".png")
fig.show()

### N 1s

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/Ag5 and Br4TPP/"
Folders=['0001_230323_AQ']
Files=["N 1s_RT_150_400"]  

names=["30 °C", "150 °C", "400 °C"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
        
    x_len=XPS.shape[0]//NumberOfSpectra
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)
    XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])

    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        if(i==2):
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]))-0.2,
                                 name=names[i]))
        else:  
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Au(111)_Spectrum_N_RT_150C_400C"+".svg")
fig.write_image(Path_s+"/"+ "Au(111)_Spectrum_N_RT_150C_400C"+".png")
fig.show()

### Ag 3d

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/Ag5 and Br4TPP/"
Folders=['0001_230323_AQ']
Files=["Ag 3d_RT_150_400"]  

names=["30 °C", "150 °C", "400 °C"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.4))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    #XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)
    XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        
        if (i==2):
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]))/10,
                                 name=names[i]))
        else:
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_xaxes(autorange = "reversed")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Au(111)_Spectrum_Ag_RT_150C_400C"+".svg")
fig.write_image(Path_s+"/"+ "Au(111)_Spectrum_Ag_RT_150C_400C"+".png")
fig.show()

## Mica

### 400 C 27_04_2023

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Ag5/Ag5 and Br4TPP/Mica/"+"0006_270423_AQ"
#File of the temperature ramp
FileTR="TemperatureRamp"

#Files with XPS Data
files=["Br3p","Ag3d","Br3p","C1s","Al2p"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Mask= TemperatureRamp['Temperature'] >= 40
#TemperatureRamp=TemperatureRamp[Mask]
#TemperatureRamp['Time']=TemperatureRamp['Time']-TemperatureRamp['Time'][TemperatureRamp['Time'].index[0]]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature']+2)

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)    

    if (FileXPS=="Ag3d"):
        Mask=np.logical_and(XPS["Binding Energy"]<370, XPS["Binding Energy"]>366)
        XPS=XPS[Mask]
        XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    if (FileXPS=="Br3p"):
        Mask=np.logical_and(XPS["Binding Energy"]<186, XPS["Binding Energy"]>178.5)
        XPS=XPS[Mask]
        XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data

    print(FileXPS)
    
    x_len=XPS.shape[0]//NumberOfSpectra

    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'thermal'))
    #fig.data[0].update(zmin=-200, zmax=475)
###

    Aesthetics(fig)
    
    if (FileXPS=="Br3p"):
        fig.update_xaxes(range = [185,180])
        fig.data[0].update(zmin=100, zmax=1000)
        
    if (FileXPS=="Ag3d"):
        fig.update_xaxes(range = [370,366])


    fig.update_traces(showscale=False)#hides the colorbar
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")
    fig.write_image(FolderTR+"/"+files[j]+".png")

#### Br 3p - X-ray induced debromination

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/Ag5 and Br4TPP/Mica/"
Folders=['0006_270423_AQ']
Files=["Br 3p_X-ray damage"]  

names=["Before", "After"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.8,xanchor="right",x=0.4))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i]+4,
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]),
                                 name=names[i]))
    
    
fig.add_vline(x=184.0, line_dash="dash",annotation_text="Br-C")
fig.add_vline(x=181.7, line_dash="dash",annotation_text="Br-Ag")

fig.add_vline(x=191.5, line_dash="dash",annotation_text="Br-C")
fig.add_vline(x=188.3, line_dash="dash",annotation_text="Br-Ag")


fig.update_layout(xaxis = dict(range = [194,176]))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Br_x-ray damage"+".svg")
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Br_x-ray damage"+".png")
fig.show()

#### Br 3p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/Ag5 and Br4TPP/Mica/"
Folders=['0006_270423_AQ']
Files=["Br 3p_Xray_RT_150C_400C"]  

names=["30 °C", "150 °C", "400 °C","First measurement"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Normalised counts')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.7))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    #Mask=np.logical_and(XPS["Binding Energy"]>179, XPS["Binding Energy"] <188)
    #XPS=XPS[Mask]
    
    #if (Files[j]=="Br3p_150C_NoCu"):
    #XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        if (i==3):
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i]+4,
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]),
                                 name=names[i]))
        else:  
            fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]),
                                 name=names[i]))
    
    

fig.update_layout(xaxis = dict(range = [194,176]))
fig.add_vline(x=184.0, line_dash="dash",annotation_text="Br-C")
fig.add_vline(x=181.7, line_dash="dash",annotation_text="Br-Ag")

fig.add_vline(x=191.5, line_dash="dash",annotation_text="Br-C")
fig.add_vline(x=188.3, line_dash="dash",annotation_text="Br-Ag")
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Br_RT_150C_400C"+".svg")
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Br_RT_150C_400C"+".png")
fig.show()

#### Ag 3d

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/Ag5 and Br4TPP/Mica/"
Folders=['0006_270423_AQ']
Files=["Ag 3d_RT_150C_400C"]  

names=["30 °C", "150 °C", "400 °C"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.4))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    Mask=np.logical_and(XPS["Binding Energy"]>364, XPS["Binding Energy"] <372)
    XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)
    XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_layout(xaxis = dict(range = [372,364]))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Ag_RT_150C_400C"+".svg")
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Ag_RT_150C_400C"+".png")
fig.show()

#### C 1s

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Ag5/Ag5 and Br4TPP/Mica/"
Folders=['0006_270423_AQ']
Files=["C 1s_RT_150C_400C"]  

names=["30 °C", "150 °C", "400 °C"]
#titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.4))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    Mask=np.logical_and(XPS["Binding Energy"]>280, XPS["Binding Energy"] <291)
    XPS=XPS[Mask]
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    for i in range(0,len(names)):
        XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-=np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])
        fig.add_trace(go.Scatter(mode='lines',# marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
    

fig.update_layout(xaxis = dict(range = [291,281]))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF/Ag5"     
fig.write_image(Path_s+"/"+ "Mica_Spectrum_C_RT_150C_400C"+".svg")
fig.write_image(Path_s+"/"+ "Mica_Spectrum_C_RT_150C_400C"+".png")
fig.show()

# Thesis

## HOPG

### No Cu5

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"0013_180320_SPJ_LF"
#File of the temperature ramp
FileTR="Datalog_Br4TPP_HOPG_TP_2"

#Files with XPS Data
files=["N1s", "Br3p", "C1s"]
#Number of data per spectrum
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <= 18.7
TemperatureRamp=TemperatureRamp[Mask]
#TemperatureRamp['Time']=TemperatureRamp['Time']-TemperatureRamp['Time'][TemperatureRamp['Time'].index[0]]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    #DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    #NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
 
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)    

        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data

    print(FileXPS)
    
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra


    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'thermal'))
    #fig.data[0].update(zmin=-200, zmax=475)
###
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=500)
    
    
    if (files[j]=="Br3p"):
        fig.update_layout(xaxis = dict(range = [193,178]))
    if (files[j]=="N1s"):
        fig.update_layout(xaxis = dict(range = [404,396]))
        fig.data[0].update(zmin=100, zmax=2000)
    else:
        fig.update_layout(xaxis = dict(autorange = "reversed")) 


    fig.update_traces(showscale=False)#hides the colorbar
    Aesthetics(fig)
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")
    Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
    fig.write_image(Path_s+"/"+ "TP-XPS_HOPG_No Cu "+files[j]+".svg")
    fig.write_image(Path_s+"/"+ "TP-XPS_HOPG_No Cu "+files[j]+".png")

### Cu5

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"Br4TPP/HOPG/Reply to referees/"+"0009_101122_AQ"
#File of the temperature ramp
FileTR="Temperature Ramp"

#Files with XPS Data
files=["Cu 2p", "C 1s", "Br 3p", "N 1s"]
#Number of data per spectrum
#DataPerSpectra=[409,189,209]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <=16.85
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
 
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Thermal'))
    #fig.data[0].update(zmin=-200, zmax=475)
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=500)
    
        
    if (files[j]=="Br 3p"):
        fig.update_layout(xaxis = dict(range = [193,178]))
        fig.data[0].update(zmin=100, zmax=1000)
    if (files[j]=="N 1s"):
        fig.update_layout(xaxis = dict(range = [404,396]))
    if (files[j]=="Cu 2p"):
        fig.update_layout(xaxis = dict(range = [935,930])) 
    if (files[j]=="C 1s"):
        fig.update_layout(xaxis = dict(range = [286,282])) 


    fig.update_traces(showscale=False)#hides the colorbar
    Aesthetics(fig)
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")
    Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
    fig.write_image(Path_s+"/"+ "TP-XPS_HOPG_Cu "+files[j]+ "_zoomed in"+".svg")
    fig.write_image(Path_s+"/"+ "TP-XPS_HOPG_Cu "+ files[j]+"_zoomed in"+".png")

### XPS peaks comparison

#### Br 3p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
Folders=['0013_180320_SPJ_LF','Br4TPP/HOPG/Reply to referees/0009_101122_AQ']
Files=["Br3p_125C_NoCu","Br3p_125C_Cu"]  

names=["without Cu<sub>5", "with Cu<sub>5"]
titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.25))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)

    XPS=xl.RemoveLinearBackgorund(XPS, Points=50, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    
    
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]), 
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[0],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0],
                                 name=names[j]))
    
    

fig.update_layout(xaxis = dict(range = [194,178]))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
fig.write_image(Path_s+"/"+ "HOPG_Spectrum_Br_125C"+".svg")
fig.write_image(Path_s+"/"+ "HOPG_Spectrum_Br_125C" +".png")
fig.show()

#### N 1s

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
Folders=['0013_180320_SPJ_LF','Br4TPP/HOPG/Reply to referees/0009_101122_AQ','Br4TPP/HOPG/Reply to referees/0009_101122_AQ']
Files=["N1s_400C_NoCu", "N1s_RT_Cu","N1s_400C_Cu"]  

names=["without <br>Cu<sub>5", "with Cu<sub>5</sub> <br> 30 °C", "with Cu<sub>5</sub> <br> 400 °C"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.38))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)

    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    if (Files[j]=="N1s_400C_NoCu"):

        fig.add_trace(go.Scatter(mode='lines', marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[0],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]))/10,
                                 name=names[j]))
    
    if (Files[j]!="N1s_400C_NoCu"):
         fig.add_trace(go.Scatter(mode='lines', marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[0],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0])),
                                 name=names[j]))
    
   
    
    
fig.add_vline(x=398.3, line_dash="dash",annotation_text="=N-")
fig.add_vline(x=400.3, line_dash="dash",annotation_text="-NH-")
  
    

fig.update_layout(xaxis = dict(range = [404,395]))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
fig.write_image(Path_s+"/"+ "HOPG_Spectrum_N1s_400C"+".svg")
fig.write_image(Path_s+"/"+ "HOPG_Spectrum_N1s_400C"+".png")
fig.show()

#### Cu 2p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
Folders=['Br4TPP/HOPG/Reply to referees/0009_101122_AQ']
Files=["Cu2p_RT_150C_400C"]  

names=["30 °C", "125 °C","400 °C"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)

    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)   
    
    
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[0], marker=dict(size=10, color=colors[0]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[0],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0])),
                                 name=names[0]))
    
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[1], marker=dict(size=10, color=colors[1]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[1],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[1]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[1])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[1])),
                                 name=names[1]))
    
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[2], marker=dict(size=10, color=colors[2]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[2],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[2]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[2])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[2])),
                                 name=names[2]))
    
    
    

fig.update_layout(xaxis = dict(range = [938,928]))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
fig.write_image(Path_s+"/"+ "HOPG_Spectrum_Cu2p_30C_125C"+".svg")
fig.write_image(Path_s+"/"+ "HOPG_Spectrum_Cu2p_30C_125C"+".png")
fig.show()

## Mica

### No Cu5

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Br4TPP/Mica/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"0020_300621_AQ"
#File of the temperature ramp
FileTR="TemperatureRamp"

#Files with XPS Data
files=["Br3p"]
#Number of data per spectrum
DataPerSpectra=[169,239,309,149]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <= 17.6
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
 
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Thermal'))
    
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    
        
    fig.update_layout(xaxis = dict(range = [188,179]))

    fig.update_traces(showscale=False)#hides the colorbar
    Aesthetics(fig)
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")
    Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
    fig.write_image(Path_s+"/"+ "TP-XPS_Mica_No Cu "+files[j]+".svg")
    fig.write_image(Path_s+"/"+ "TP-XPS_Mica_No Cu "+ files[j]+".png")

### Cu5

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Br4TPP/Mica/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"0003_070721_AQ"
#File of the temperature ramp
FileTR="TemperatureRamp"

#Files with XPS Data
files=["Br3p", "Cu2p"]
#Number of data per spectrum
DataPerSpectra=[169,239,309,149]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= np.logical_and(TemperatureRamp['Time'] <= 20.1, TemperatureRamp['Time'] >= 1) 
TemperatureRamp=TemperatureRamp[Mask]
TemperatureRamp['Time']= TemperatureRamp['Time']-TemperatureRamp['Time'][3601]

regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])
TemperatureRamp['Time'][3601]


In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
 
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)
    
    if (files[j]=="Br3p"):
        XPS["Binding Energy"]+=4
        
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Thermal'))
    
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    
    if (files[j]=="Br3p"):
        fig.update_layout(xaxis = dict(range = [190,183]))
        fig.data[0].update(zmin=-100, zmax=400)
    if (files[j]=="Cu2p"):
        fig.update_layout(xaxis = dict(range = [936,930])) 
        

    fig.update_traces(showscale=False)#hides the colorbar
    Aesthetics(fig)
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")
    Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
    fig.write_image(Path_s+"/"+ "TP-XPS_Mica_Cu "+files[j]+ "_new" + ".svg")
    fig.write_image(Path_s+"/"+ "TP-XPS_Mica_Cu "+files[j]+ "_new" + ".png")

#### C 1s

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Br4TPP/Mica/Reply to referees/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"0000_011122_AQ"
#File of the temperature ramp
FileTR="Temperature Ramp"

#Files with XPS Data
files=["C 1s"]
#Number of data per spectrum
DataPerSpectra=[169,239,309,149]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = xl.ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
xl.AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
Mask= TemperatureRamp['Time'] <= 15.7
TemperatureRamp=TemperatureRamp[Mask]
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

In [ ]:
for j in range(0,len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
 
    XPS=xl.ReadXPSData(FolderTR, FileXPS)

    #number of spectra composing the dataset
    DataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns
    NumberOfSpectra=(XPS.shape[0]//DataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=xl.TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    
    #Remove the linear background and print the data
    XPS=xl.RemoveLinearBackgorund(XPS, Points=15, NumberOfSpectra=NumberOfSpectra)
        
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra

    print(FileXPS)        
        
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len),#-XPS["Background CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Thermal'))
    
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)
    
    
    fig.data[0].update(zmin=-500, zmax=10000)
    fig.update_layout(xaxis = dict(range = [287,283]))
        

    fig.update_traces(showscale=False)#hides the colorbar
    Aesthetics(fig)
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+files[j]+".svg")
    Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
    fig.write_image(Path_s+"/"+ "TP-XPS_Mica_Cu "+files[j]+ "_new" + ".svg")
    fig.write_image(Path_s+"/"+ "TP-XPS_Mica_Cu "+files[j]+ "_new" + ".png")

### XPS peaks comparison

#### Br 3p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Br4TPP/Mica/"
Folders=['0020_300621_AQ','0003_070721_AQ']
Files=["Br3p_150C_NoCu","Br3p_150C_Cu"]  

names=["without Cu<sub>5", "with Cu<sub>5"]
titles=["Cu5 HOPG Cu 2p"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.4))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
        
    
    Mask=np.logical_and(XPS["Binding Energy"]>179, XPS["Binding Energy"] <188)
    XPS=XPS[Mask]
    
    #if (Files[j]=="Br3p_150C_NoCu"):
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    
    
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[j], marker=dict(size=10, color=colors[j]),
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[0],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0]),
                                 name=names[j]))
    
    

fig.update_layout(xaxis = dict(range = [188,179]))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Br_150C"+".svg")
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Br_150C"+".png")
fig.show()

#### Cu 2p

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
Folders=['Br4TPP/Mica/0003_070721_AQ/']
Files=["Cu2p_50C_150C_400C"]  

names=["30 °C", "150 °C", "400 °C"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
#fig.update_layout(xaxis = dict(autorange = "reversed"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.35))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
    Mask=np.logical_and(XPS["Binding Energy"]>928, XPS["Binding Energy"] <941)
    XPS=XPS[Mask]

    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)

    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)   
    
    for i in range(0, NumberOfSpectra):
        fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[i], marker=dict(size=10, color=colors[i]), opacity=0.5,
                                     x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                     y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                     name=names[i]))
  
fig.update_layout(xaxis = dict(range =[941,928] ))
Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Cu2p_30C_150C_400C"+".svg")
fig.write_image(Path_s+"/"+ "Mica_Spectrum_Cu2p_30C_150C_400C"+".png")
fig.show()

### Cyclodehydrogenation

In [ ]:
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/Br4TPP/Mica/Reply to referees/"
Folders=['0000_011122_AQ']
Files=["C1s_cyclodehydrogenation_RT_150C_400C"]  

names=["30 °C", "150 °C", "400 °C"]

symbol=["circle","cross","diamond"]
colors=["blue", "red", "green"]

length=168 #number of rows of each spectrum including the headers
DataPerSpectra=np.zeros(len(Files))
for i in range (len(Files)):
    DataPerSpectra[i]=length
HeaderColumns=8 #number of columns comprising the headers

In [ ]:
fig = go.Figure()

fig.update_layout(template="plotly_white")
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Intensity (a.u.)')
fig.update_yaxes(showticklabels=False,title_standoff = 30)
fig.update_layout(yaxis = dict( tickfont = dict(size = 30)), xaxis = dict( tickfont = dict(size = 30)))
fig.update_layout(font=dict(size=30))
#add the black box around the graph
fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
#add black ticks
fig.update_xaxes(showgrid=True,ticks="inside",tickwidth=4)
fig.update_yaxes(showgrid=True,ticks="inside",tickwidth=4)
    
fig.update_layout(legend=dict(yanchor="top",y=0.79,xanchor="right",x=0.35))
fig.update_layout(margin=go.layout.Margin(l=0,r=0,b=20,t=0)) #t=50
fig.update_layout(autosize=False, width=550, height=500)

for j in range(0,len(Files)):
    
    FolderTR=Path+Folders[j]
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers
    FileXPS=Files[j]
    NumericDataPerSpectrum=xl.Count_DataPerSpectrum(FolderTR + "/"+ FileXPS)-HeaderColumns #number of rows with the numeric data
    #Read the XPS data and assign them to a pandas dataframe
    XPS=xl.ReadXPSData(FolderTR, FileXPS)
    #number of spectra composing the dataset
    NumberOfSpectra=int(XPS.shape[0]//NumericDataPerSpectrum)
    
   
    XPS=xl.RemoveLinearBackgorund(XPS, Points=10, NumberOfSpectra=NumberOfSpectra)
    
    x_len=XPS.shape[0]//NumberOfSpectra
    
    print(x_len)
    print(NumberOfSpectra)
    
    
    for i in range (0, NumberOfSpectra):
        fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[i], marker=dict(size=10, color=colors[i]), opacity=0.5,
                                 x=XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i],
                                 y=XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]/(np.max(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])-np.min(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i])),
                                 name=names[i]))
        
fig.add_vline(x=290, line_dash="dash",annotation_text="π - π<sup>*")
fig.add_vline(x=288.6, line_dash="dash",annotation_text="C-Br <br> C-NH")


    
    
fig.update_layout(xaxis = dict(range =[291,282]))

Path_s="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Thesis/Chapters/COF"     
fig.write_image(Path_s+"/"+ "Mica_Cyclodehydrogenation"+".svg")
fig.write_image(Path_s+"/"+ "Mica_Cyclodehydrogenation"+".png")
fig.show()